# Simple PAWPULARITY convnet

Based on https://keras.io/examples/vision/mnist_convnet/ by [fchollet](https://twitter.com/fchollet)


## Setup

In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import os
import tqdm

In [ ]:
SEED = 123
NROWS = None #500
IMAGE_SIZE = (128, 128)

if 'KAGGLE_URL_BASE' in os.environ:
    TRAIN_IMAGES_DIR = "../input/petfinder-pawpularity-score/train"
    TRAIN_DS = "../input/petfinder-pawpularity-score/train.csv"
    TEST_IMAGES_DIR = "../input/petfinder-pawpularity-score/test"
    TEST_DS = "../input/petfinder-pawpularity-score/test.csv"
    SUBMISSION_IMAGES_DIR = "../input/petfinder-pawpularity-score/test"
    SUBMISSION_DS = "../input/petfinder-pawpularity-score/test.csv"
else:
    TRAIN_IMAGES_DIR = "data/sub/train"
    TRAIN_DS = "data/sub/train.csv"
    TEST_IMAGES_DIR = "data/sub/test"
    TEST_DS = "data/sub/test.csv"
    SUBMISSION_IMAGES_DIR = "data/test"
    SUBMISSION_DS = "data/test.csv"

INPUT_SHAPE = (*IMAGE_SIZE, 3)
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.1
EPOCHS = 15

In [ ]:
!nvidia-smi

In [ ]:
def read_images(img_dir, dataset):
    images = [None] * len(dataset)
    for index, img_id in enumerate(tqdm.tqdm(list(dataset.Id))):
        img = keras.preprocessing.image.load_img(f"{img_dir}/{img_id}.jpg", 
                                                 target_size=IMAGE_SIZE,
                                                 interpolation="bilinear")
        #img = keras.preprocessing.image.load_img(f"{img_dir}/{img_id}.jpg")
        #img = keras.preprocessing.image.smart_resize(img, IMAGE_SIZE)
        img = keras.preprocessing.image.img_to_array(img) / 255.
        images[index] = img
    return np.array(images)

## Prepare the data

In [ ]:
train_ds = pd.read_csv(TRAIN_DS, nrows=NROWS)
train_img = read_images(TRAIN_IMAGES_DIR, train_ds)
train_img.shape

In [ ]:
plt.imshow(train_img[99])

## Build the model

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=INPUT_SHAPE),
#        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
#        layers.MaxPooling2D(pool_size=(2, 2)),
#        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
#        layers.MaxPooling2D(pool_size=(2, 2)),
#        layers.Conv2D(64, kernel_size=(5, 5), activation="relu"),
#        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(256, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(5, 5), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(512, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(1, activation="linear"),
    ]
)

model.summary()

## Train the model

In [ ]:
x_train = train_img
y_train = train_ds.Pawpularity.values
optimizer = keras.optimizers.Adam()
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor="val_root_mean_squared_error",
    patience=2,
    factor=0.1,
    verbose=1
)

model.compile(loss="mse", 
              optimizer="adam",
              metrics=[keras.metrics.RootMeanSquaredError()])

In [ ]:
model.fit(x_train, y_train, 
          batch_size=BATCH_SIZE, 
          epochs=EPOCHS, 
          callbacks=[reduce_lr],
          validation_split=VALIDATION_SPLIT)

## Evaluate the trained model

In [ ]:
test_ds = pd.read_csv(TEST_DS)
if "Pawpularity" in test_ds.columns:
    #Si no existe es que no es DS de evaluacion sino el de submission
    test_images = read_images(TEST_IMAGES_DIR, test_ds)
    model.evaluate(test_images, test_ds.Pawpularity.values)

# Submission

In [ ]:
subms_ds = pd.read_csv(SUBMISSION_DS)
subms_img = read_images(SUBMISSION_IMAGES_DIR, subms_ds)
predictions = model.predict(subms_img)
predictions[:5]

In [ ]:
subms_ds["Pawpularity"] = predictions
subms_ds[["Id", "Pawpularity"]].to_csv("submission.csv", index=False)